# **Prepocessing**

Preprocessing adalah tahap pra-pemrosesan data sebelum data digunakan untuk analisis, pemodelan, atau pelatihan algoritma machine learning.
Tujuannya adalah agar data menjadi bersih, konsisten, dan siap dipakai, sehingga model bisa bekerja lebih baik dan hasilnya lebih akurat.

## **Mengapa Preprocessing Penting?**

1. Data mentah biasanya tidak rapi (ada nilai kosong, duplikasi, outlier, format berbeda).

2. Algoritma machine learning sensitif terhadap skala, tipe data, dan distribusi data.

3. Preprocessing membuat data:

    * Lebih seragam

    * Lebih mudah dipahami oleh algoritma

    * Mengurangi bias atau error

## Langkah-langkah Preprocessing (umum)

1. Data Cleaning (Pembersihan Data)

    * Menghapus duplikasi data

    * Menangani nilai kosong (missing value)

    * Menghapus atau memperlakukan outlier

2. Data Transformation (Transformasi Data)

    * Normalisasi atau standarisasi skala (contoh: semua fitur punya skala 0–1 atau rata-rata 0, standar deviasi 1)

    * Encoding data kategorikal ke bentuk numerik (contoh: Male=0, Female=1)

3. Data Reduction (Reduksi Data)

    * Mengurangi dimensi (misalnya PCA)

    * Menghapus fitur yang tidak relevan

### **1. Pengumpulan Data**

In [1]:
import pandas as pd
from sqlalchemy import create_engine

MYSQL_HOST = "mysql-3634ef1a-tugas-pendata.g.aivencloud.com"
MYSQL_PORT = 17416
MYSQL_USER = "avnadmin"
MYSQL_PASSWORD = "AVNS_2NRSFWfr9pGMEI7BSpA"
MYSQL_DATABASE = "defaultdb"

PG_HOST = "pg-359aec68-tugas-pendata.g.aivencloud.com"
PG_PORT = 17416
PG_USER = "avnadmin"
PG_PASSWORD = "AVNS_oal2yP3mG6JLIwX3BUK"
PG_DATABASE = "defaultdb"

mysql_engine = create_engine(
    f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DATABASE}"
)

pg_engine = create_engine(
    f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DATABASE}"
)

mysql_query = "SELECT * FROM iris_data;"
df_mysql = pd.read_sql(mysql_query, mysql_engine)

pg_query = "SELECT * FROM iris_data;"
df_postgres = pd.read_sql(pg_query, pg_engine)

print("Data dari MySQL:")
print(df_mysql.head())
 
print("\nData dari PostgreSQL:")
print(df_postgres.head())


Data dari MySQL:
   id        class  petal_length  petal_width
0   1  Iris-setosa           1.4          0.2
1   2  Iris-setosa           1.4          0.2
2   3  Iris-setosa           1.3          0.2
3   4  Iris-setosa           1.5          0.2
4   5  Iris-setosa           1.4          0.2

Data dari PostgreSQL:
   id        class  sepal_length  sepal_width
0   1  Iris-setosa           5.1          3.5
1   2  Iris-setosa           4.9          3.0
2   3  Iris-setosa           4.7          3.2
3   4  Iris-setosa           4.6          3.1
4   5  Iris-setosa           5.0          3.6


In [2]:
import os
mysql_query = "SELECT * FROM iris_data;"
df_mysql = pd.read_sql(mysql_query, mysql_engine)

pg_query = "SELECT * FROM iris_data;"
df_postgres = pd.read_sql(pg_query, pg_engine)


#Menghapus file yang di upload sebelumnya
if os.path.exists(".env"):
    os.remove(".env")

df_combined = pd.merge(df_postgres, df_mysql.drop(columns=['class']), on="id", how="inner")

print("\nData gabungan:")
print(df_combined)


Data gabungan:
      id           class  sepal_length  sepal_width  petal_length  petal_width
0      1     Iris-setosa           5.1          3.5           1.4          0.2
1      2     Iris-setosa           4.9          3.0           1.4          0.2
2      3     Iris-setosa           4.7          3.2           1.3          0.2
3      4     Iris-setosa           4.6          3.1           1.5          0.2
4      5     Iris-setosa           5.0          3.6           1.4          0.2
..   ...             ...           ...          ...           ...          ...
145  146  Iris-virginica           6.7          3.0           5.2          2.3
146  147  Iris-virginica           6.3          2.5           5.0          1.9
147  148  Iris-virginica           6.5          3.0           5.2          2.0
148  149  Iris-virginica           6.2          3.4           5.4          2.3
149  150  Iris-virginica           5.9          3.0           5.1          1.8

[150 rows x 6 columns]


## Pembersihan Outlier

**Seleksi outlier menggunakan metode ABOD**

In [ ]:
from pycaret.datasets import get_data
from pycaret.anomaly import *

# Use only the feature columns for anomaly detection
anomaly = df_combined[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

exp_name = setup(data=anomaly)
abod = create_model('abod', fraction=0.05)
abod_df = assign_model(abod) 

outlier_df = abod_df[abod_df['Anomaly'] == 0]
outlier_df

,Description,Value
0,Session id,6177
1,Original data shape,"(150, 4)"
2,Transformed data shape,"(150, 4)"
3,Numeric features,4
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


,sepal_length,sepal_width,petal_length,petal_width,Anomaly,Anomaly_Score
0,5.1,3.5,1.4,0.2,0,-556.251421
1,4.9,3.0,1.4,0.2,0,-400.000928
2,4.7,3.2,1.3,0.2,0,-93.421993
3,4.6,3.1,1.5,0.2,0,-99.229221
4,5.0,3.6,1.4,0.2,0,-82.176201
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,-13.831461
146,6.3,2.5,5.0,1.9,0,-9.110201
147,6.5,3.0,5.2,2.0,0,-20.571707
148,6.2,3.4,5.4,2.3,0,-6.345400


* seharusnya pada data diatas hanya tersisa 142 data karean code diatas mendeteksi anomaly=0

In [4]:
# Simpan hasil ke file CSV
outlier_df.to_csv("outlier_abod.csv", index=False)


Tapi pada csv diatas sudah bersih 142 data

**Seleksi outlier menggunakan metode KNN**

In [7]:
from pycaret.datasets import get_data
from pycaret.anomaly import *

# Use only the feature columns for anomaly detection
anomaly = df_combined[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

exp_name = setup(data=anomaly)
knn = create_model('knn', fraction=0.05)
knn_df = assign_model(knn) 

knn_outlier_df = knn_df[abod_df['Anomaly'] == 0]
knn_outlier_df

,Description,Value
0,Session id,211
1,Original data shape,"(150, 4)"
2,Transformed data shape,"(150, 4)"
3,Numeric features,4
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


,sepal_length,sepal_width,petal_length,petal_width,Anomaly,Anomaly_Score
0,5.1,3.5,1.4,0.2,0,0.141421
1,4.9,3.0,1.4,0.2,0,0.173205
2,4.7,3.2,1.3,0.2,0,0.264575
3,4.6,3.1,1.5,0.2,0,0.264575
4,5.0,3.6,1.4,0.2,0,0.244949
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0.374166
146,6.3,2.5,5.0,1.9,0,0.479583
147,6.5,3.0,5.2,2.0,0,0.387298
148,6.2,3.4,5.4,2.3,0,0.624500


In [8]:
# Simpan hasil ke file CSV
knn_outlier_df.to_csv("outlier_knn.csv", index=False)


**Seleksi outlier menggunakan metode LOF**

In [9]:
from pycaret.datasets import get_data
from pycaret.anomaly import *

# Use only the feature columns for anomaly detection
anomaly = df_combined[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]

exp_name = setup(data=anomaly)
lof = create_model('lof', fraction=0.05)
lof_df = assign_model(lof) 

lof_outlier_df = lof_df[abod_df['Anomaly'] == 0]
lof_outlier_df

,Description,Value
0,Session id,4701
1,Original data shape,"(150, 4)"
2,Transformed data shape,"(150, 4)"
3,Numeric features,4
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


,sepal_length,sepal_width,petal_length,petal_width,Anomaly,Anomaly_Score
0,5.1,3.5,1.4,0.2,0,0.976302
1,4.9,3.0,1.4,0.2,0,1.008758
2,4.7,3.2,1.3,0.2,0,1.019841
3,4.6,3.1,1.5,0.2,0,1.049882
4,5.0,3.6,1.4,0.2,0,0.958473
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0.978474
146,6.3,2.5,5.0,1.9,0,1.004232
147,6.5,3.0,5.2,2.0,0,0.980847
148,6.2,3.4,5.4,2.3,0,1.021819


In [11]:
# Simpan hasil ke file CSV
lof_outlier_df.to_csv("outlier_lof.csv", index=False)


**Data sudah dibersihkan dari outlier dan sudah mendapatkan 3 dataset baru dari pembersihan menggunakan 3 metode**